# Tribolium embryo morphometry
Authors: Robert Haase, Daniela Vorkel, June 2020

This is the pyclesperanto version of a workflow earlier [published for clij2](https://clij.github.io/clij2-docs/md/tribolium_morphometry/). 
[ImageJ Macro original](https://github.com/clij/clij2-docs/tree/master/src/main/macro/tribolium_morphometry.ijm)

This script is an example of heavy GPU-accelerated processing. It is recommended to use a dedicated
graphics card with at least 8 GB of GDDR6 memory. Otherwise, it may be quite slow.

Let's start by checking that pyclesperanto is installed and which GPU it uses.

In [1]:
import pyclesperanto_prototype as cle

cle.select_device('RTX')

<GeForce RTX 2080 Ti on Platform: NVIDIA CUDA (1 refs)>

## Load a data set
The dataset is available [online](https://git.mpi-cbg.de/rhaase/clij2_example_data/blob/master/lund1051_resampled.tif).
It shows a *Tribolium castaneum* embryo, imaged by a custom light sheet microscope, at a wavelength of 488nm (Imaging credits: Daniela Vorkel, Myers lab, MPI CBG). 
The data set has been resampled to a voxel size of 1x1x1 microns. The embryo expresses nuclei-GFP. We will use the dataset to detect nuclei and to generate an estimated cell-segmentation.

All processing steps are performed in 3D space. For visualization purpose, we are using the maximum intensity projection in Z: 

In [2]:
from skimage.io import imread

image = imread('C:/structure/data/lund1051_resampled.tif')

# print out the spatial dimensions of the image
print(image.shape)

FileNotFoundError: [Errno 2] No such file or directory: 'C:\\structure\\data\\lund1051_resampled.tif'

In [ ]:
import matplotlib.pyplot as plt

# convenience function for visualisation
def show(gpu_image):
    if (len(gpu_image.shape) == 3):
        gpu_max_projection = cle.create_2d_yx(gpu_image)
        cle.maximum_z_projection(gpu_image, gpu_max_projection)
        max_projection = cle.pull(gpu_max_projection)
        plt.imshow(max_projection)
    else:
        plt.imshow(cle.pull(gpu_image))
        
    plt.show()

def show_binary(gpu_image):
    if (len(gpu_image.shape) == 3):
        gpu_max_projection = cle.create_2d_yx(gpu_image)
        cle.maximum_z_projection(gpu_image, gpu_max_projection)
        max_projection = cle.pull(gpu_max_projection)
        plt.imshow(max_projection, vmin=0, vmax=1)
    else:
        plt.imshow(cle.pull(gpu_image), vmin=0, vmax=1)
        
    plt.show()

In [ ]:
# push image to GPU memory and show it
gpu_input = cle.push_zyx(image)

show(gpu_input)

## Spot detection
After some noise removal/smoothing, we perform a local maximum detection:

In [ ]:
# gaussian blur
sigma = 2.0
gpu_blurred = cle.gaussian_blur(gpu_input, sigma_x=sigma, sigma_y=sigma, sigma_z=sigma)

# detect maxima
gpu_detected_maxima = cle.detect_maxima_box(gpu_blurred)
show(gpu_detected_maxima)

## Spot curation
Now, we remove spots with values below a certain intensity and label the remaining spots.

In [ ]:
# threshold
threshold = 300.0
gpu_thresholded = cle.greater_constant(gpu_blurred, constant=threshold)

# mask
gpu_masked_spots = cle.mask(gpu_detected_maxima, gpu_thresholded)

# label spots
gpu_labelled_spots = cle.connected_components_labeling_box(gpu_masked_spots)

Let's see how many spots are left:

In [ ]:
number_of_spots = cle.maximum_of_all_pixels(gpu_labelled_spots)
print("Number of detected spots: " + str(number_of_spots))

## Expanding labelled spots
Next, we spatially extend the labelled spots by applying a maximum filter.

In [ ]:
# label map closing
number_of_dilations = 10
number_of_erosions = 4

flip = cle.create_like(gpu_labelled_spots)
flop = cle.create_like(gpu_labelled_spots)
flag = cle.create([1,1,1])
cle.copy(gpu_labelled_spots, flip)

for i in range (0, number_of_dilations) :
    cle.onlyzero_overwrite_maximum_box(flip, flag, flop)
    cle.onlyzero_overwrite_maximum_diamond(flop, flag, flip)
    if (i % 2 == 0):
        show(flip)

Afterwards, we erode all labels in the map and get a final result of cell segementation.

In [ ]:
flap = cle.greater_constant(flip, constant = 1)
for i in range(0, number_of_erosions):
    cle.erode_box(flap, flop)
    cle.erode_box(flop, flap)

gpu_labels = cle.mask(flip, flap)
show(gpu_labels)

## Draw connectivity of the cells as a mesh
We then read out all current positions of detected nuclei as a pointlist to generate 
a distance matrix of all nuclei towards each other:

In [ ]:
gpu_pointlist = cle.labelled_spots_to_pointlist(gpu_labelled_spots)
gpu_distance_matrix = cle.generate_distance_matrix(gpu_pointlist, gpu_pointlist)
show(gpu_distance_matrix)

Starting from the label map of segmented cells, we generate a touch matrix:

In [ ]:
gpu_touch_matrix = cle.generate_touch_matrix(gpu_labels)

# touch matrix:
# set the first column to zero to ignore all spots touching the background (background label 0, first column)
cle.set_column(gpu_touch_matrix, 0, 0)

show(gpu_touch_matrix)

Using element by element multiplication of a distance matrix and a touch matrix, we calculate the length of 
each edge. We use this result to draw a mesh with a color gradient of distance (between 0 and 50 micron):

In [ ]:
gpu_touch_matrix_with_distances = cle.multiply_images(gpu_touch_matrix, gpu_distance_matrix);

gpu_mesh = cle.create_like(gpu_input)
cle.set(gpu_mesh, 0)

cle.touch_matrix_to_mesh(gpu_pointlist, gpu_touch_matrix_with_distances, gpu_mesh);

show(gpu_mesh);